# Task

Captura una o varias imágenes con monedas no solapadas. Tras visualizar la imagen, identifica de forma interactiva (por ejemplo haciendo clic en la imagen) una moneda de un valor determinado en la imagen (por ejemplo de 1€). Tras ello, la tarea se resuelve mostrando por pantalla el número de monedas y la cantidad de dinero presentes en la imagen. No hay restricciones sobre utilizar medidas geométricas o de color. ¿Qué problemas han observado?

<ins>Nota:</ins> Para establecer la correspondencia entre píxeles y milímetros, comentar que la moneda de un euro tiene un diámetro de 23.25 mm. la de 50 céntimos de 24.35, la de 20 céntimos de 22.25, etc.

<ins>Extras:</ins> Considerar que la imagen pueda contener objetos que no son monedas y/o haya solape entre las monedas. Demo en vivo.

El mundo real es muy variado, las imágenes no siempre se capturan con unas condiciones de iluminación tan buenas o controladas. Ejemplo con aplicación de variantes de umbralizados ofrecidas por OpenCV

In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [ ]:
# Load the image
img = cv2.imread('Monedas.jpg')

# If the image width is greater than 800 pixels, reduce the size
if img.shape[1] > 800:
    factor_reduccion = img.shape[1] // 500  # Calculate reduction factor based on width
    img = cv2.resize(img, (img.shape[1] // factor_reduccion, img.shape[0] // factor_reduccion))  # Resize image

# Convert the image from BGR to RGB
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Convert the image to grayscale
img_gris = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Threshold value
umbral = 200

# Apply binary thresholding with Otsu's method (inverted binary)
th2, img_th2 = cv2.threshold(img_gris, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Apply Gaussian blur to the grayscale image
img_gaussian_blur = cv2.GaussianBlur(img_gris, (9, 9), 3)

# Detect edges using the Canny edge detector
canny_output = cv2.Canny(img_gaussian_blur, 0, 85)

# Subtract the Canny edges from the thresholded image
diff = cv2.subtract(img_th2, canny_output)

# Find contours from the difference image
contornos2, hierarchy2 = cv2.findContours(diff, 
    cv2.RETR_EXTERNAL,  # Extract only external contours
    cv2.CHAIN_APPROX_SIMPLE)  # Approximate the contour points

# Apply median blur to the grayscale image
img_blur = cv2.medianBlur(img_gris, 7)

# Create a blank image with the same shape as the original for drawing contours
img_cont = np.zeros(img_rgb.shape, dtype=np.uint8)

# Loop through all external contours
for c in contornos2:
    # Calculate the area of the contour
    area = cv2.contourArea(c)

    # Filter out small areas (optional, depending on use case)
    if area > 10:
        # Calculate the perimeter of the contour
        perimetro = cv2.arcLength(c, True)

        # Bounding rectangle aligned with the image axes
        x, y, w, h = cv2.boundingRect(c)

        # Minimum enclosing circle for the contour
        (cx, cy), radio = cv2.minEnclosingCircle(c)

        # Draw the contour filled with white color
        cv2.drawContours(img_cont, [c], -1, (255, 255, 255), -1)

# Display the original RGB image
plt.subplot(1, 4, 1)
plt.imshow(img_rgb)

# Display the Canny edge output
plt.subplot(1, 4, 2)
plt.imshow(canny_output)    

# Display the difference image
plt.subplot(1, 4, 3)
plt.imshow(diff)

# Display the contours on the blank image
plt.subplot(1, 4, 4)
plt.imshow(img_cont)

# Create a list of tuples containing the contour and its minimum enclosing circle's radius
contornos_radi = [(c, cv2.minEnclosingCircle(c)[1]) for c in contornos2]

# Placeholder for the scale relation in millimeters per pixel
relacion_mm_por_pixel = 0

In [ ]:
# Dictionary with coin diameters and their respective values (in euros)
monedas = {
    16.25: 0.01,  # 1 cent
    18.75: 0.02,  # 2 cents
    21.25: 0.05,  # 5 cents
    19.75: 0.10,  # 10 cents
    22.25: 0.20,  # 20 cents
    24.25: 0.50,  # 50 cents
    23.25: 1.00,  # 1 euro
    25.75: 2.00   # 2 euros
}

In [ ]:
def isCoin(x, y, img):
    """
    Determines whether a specific pixel location in the image indicates the presence of a coin.

    This function checks the pixel value at the specified (x, y) coordinates in the given image.
    If the pixel value is non-zero, it indicates that there is something present at that location,
    which is likely a coin. The function is used to help identify if a mouse click occurred on a 
    coin in the image.

    Parameters:
    -----------
    x : int
        The x-coordinate of the pixel in the image.
    y : int
        The y-coordinate of the pixel in the image.
    img : numpy.ndarray
        The image in which to check the pixel value, expected to be in grayscale.

    Returns:
    --------
    bool
        True if the pixel value is non-zero (indicating the presence of a coin), otherwise False.
    """
    # Check if the pixel at coordinates (x, y) in the image is non-zero
    # If the pixel value is not zero, it indicates there is something present (likely a coin)
    return (0 != np.sum(img[y, x]))

In [ ]:
def countMoney():
    """
    Calculates the total amount of money based on detected contours and their estimated diameters.
    
    The function uses a global `relacion_mm_por_pixel` variable to convert the pixel measurements
    into millimeters. It compares each contour's estimated diameter with known coin diameters from 
    the `monedas` dictionary. If the diameter matches within a specified error margin, it adds the 
    coin's value to the total and keeps a count of how many coins of each denomination were detected.

    The function prints the total amount of money detected and the count of each coin denomination.
    
    Global Variables:
    -----------------
    relacion_mm_por_pixel : float
        Conversion factor from pixels to millimeters.
    
    Variables:
    ----------
    error : float
        The allowed error margin when comparing estimated diameters with real coin diameters.
    dinero_total : float
        The total amount of money calculated.
    conteo : dict
        A dictionary that keeps track of the count of each coin denomination.
    contornos_radi : list
        A list of tuples, where each tuple contains a contour and its corresponding radius.
    monedas : dict
        A dictionary with coin diameters (in mm) as keys and their values (in euros) as values.

    Returns:
    --------
    None
        This function does not return a value. It prints the total amount and the count of each denomination.
    """
    global relacion_mm_por_pixel  # Use the global variable to define the mm per pixel ratio

    print('The mm/pixel ratio is:', relacion_mm_por_pixel)

    # Allowed error in coin diameters (in millimeters)
    error = 0.5
    dinero_total = 0  # Total amount of money

    # Dictionary to keep count of each coin denomination
    conteo = {
        0.01: 0,  # 1 cent
        0.02: 0,  # 2 cents
        0.05: 0,  # 5 cents
        0.10: 0,  # 10 cents
        0.20: 0,  # 20 cents
        0.50: 0,  # 50 cents
        1.00: 0,  # 1 euro
        2.00: 0   # 2 euros
    }

    # Loop through each contour and its corresponding radius
    for c, radio in contornos_radi:
        # Calculate the diameter of the contour in pixels
        diametro_modeda_pixeles = radio * 2

        # Uncomment these lines to print total money and estimated diameter in mm
        # print('Total money:', dinero_total)
        # print(f'This contour has an estimated diameter: {diametro_modeda_pixeles * relacion_mm_por_pixel} mm')

        # Compare the calculated diameter (in mm) with actual coin diameters
        for diametro_real, valor in monedas.items():
            if (diametro_real - error) < diametro_modeda_pixeles * relacion_mm_por_pixel < (diametro_real + error):
                # If the diameter is within the allowed error range, add the coin value to total
                dinero_total += valor
                conteo[valor] += 1  # Increment the count of this coin
                # Uncomment to print the detected coin value
                # print(f'This coin is worth {valor} euros')
                break

    # Print the total amount of money detected
    print(f'The total amount is: {round(dinero_total, 2)} euros')

    # Print the count of each coin denomination
    print('Coin count:')
    for valor, cantidad in conteo.items():
        print(f'{cantidad} coins of {valor} euros')

In [ ]:
def on_mouse(event, x, y, flags, userdata):
    """
    Mouse event handler that calculates the pixel-to-mm ratio based on a clicked coin and initiates the money counting process.
    
    When the user clicks on the window, the function checks if the click occurred inside a detected coin contour. If the click is inside
    a contour, the function calculates the coin's diameter in pixels and compares it to the real-world diameter of a 1-euro coin 
    (23.25 mm). This allows the function to compute the relationship between millimeters and pixels (`relacion_mm_por_pixel`). 
    Once the relationship is established, the function calls `countMoney()` to calculate the total amount of money.

    Parameters:
    -----------
    event : int
        The type of mouse event (e.g., left button click).
    x : int
        The x-coordinate of the mouse pointer during the event.
    y : int
        The y-coordinate of the mouse pointer during the event.
    flags : int
        Any flags passed during the event (not used in this function).
    userdata : any
        Additional user data passed to the callback function (not used in this function).

    Returns:
    --------
    None
        This function does not return a value but prints information about the coin and initiates money counting.
    """
    global relacion_mm_por_pixel  # Use global variable for the pixel-to-mm ratio

    # Check if the left mouse button was clicked and if the click is inside a coin contour
    if event == cv2.EVENT_LBUTTONDOWN and isCoin(x, y, img_cont):
        # Loop through the contours and check if the click is inside any contour
        for c, radio in contornos_radi:
            dist = cv2.pointPolygonTest(c, (x, y), False)

            if dist >= 0:  # If the click is inside the contour
                print(f'Radius in pixels: {radio}')

                # Calculate the diameter of the coin in pixels
                diametro_modeda_pixeles = radio * 2

                # Real diameter of a 1-euro coin in millimeters
                diametro_moneda_real = 23.25

                # Calculate the mm/pixel ratio
                relacion_mm_por_pixel = diametro_moneda_real / diametro_modeda_pixeles

                print(f'Mm/pixel ratio: {relacion_mm_por_pixel}')
                print("Starting to count money!")
                countMoney()  # Call the money counting function
                break

# Window setup and display
windowName = "amor"

# Create a named window
cv2.namedWindow(windowName)

# Set the mouse callback function for the window
cv2.setMouseCallback(windowName, on_mouse)

# Display the image in the window
cv2.imshow(windowName, img_rgb)

# Keep the window open until the 'Esc' key is pressed
while cv2.waitKey(20) != 27:
    dummy = 0  # Loop runs until 'Esc' key is pressed
    
# Destroy all windows once the loop ends
cv2.destroyAllWindows()